<a href="https://colab.research.google.com/github/Rchess10/hackathon_2/blob/main/Hackathon2_Groupe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sujet 2 : LLM finement ajusté pour classification de sentiments + réponses contextuelles**

**Objectif**


Créer un mini-LLM qui peut :
* Classifier une ´emotion (positif, négatif, neutre),
* Fournir une réponse basée sur des documents (type RAG).


**étapes guidées**
1. Choisir le jeu de données : par exemple IMDB (film reviews). Revoir : encoding.pdf,
Supervisé mesures perfs.pdf.
2. Chargement du modéle : utilisez distilbert ou roberta-small.
3. Fine-tuning LoRA (simple) :
• Utilisez le package peft pour faire un fine-tuning léger.
• Vous pouvez figer les poids et ne fine-tuner qu’une petite partie.
4. Création du système RAG :
* Embedding avec sentence-transformers.
* Similarité cosinus
* Génération via prompt + contexte.
* Revoir : GenAI RAG 1.pdf, GenAI RAG 2.pdf.
5. Interface simple : interface CLI pour faire une boîte de texte et afficher la réponse, si si le temps le permet, essayer Streamlit.

# **ETAPE 1**

installation des librairies

importation des modules

importation & échantillonnage des données

## etape 1.1 installation des librairies


In [ ]:
# Installation des librairies
!pip install -q torch transformers datasets accelerate peft scikit-learn numpy pandas tqdm openpyxl


## etape 1.2 importation modules

cellule à compléter au fur et mesure

In [ ]:
# importation de librairies /modules dans le script
import os  # operations fichiers / système
import json  # fichiers JSON data charger/sauvegarder
import time  # mesure latence
from datetime import datetime  # timestamps
from collections import Counter  # compter le repetitions

# Data
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer  #  TF-IDF
from sklearn.neighbors import NearestNeighbors  #  kNN search
from sklearn.metrics.pairwise import cosine_similarity  #  similarité cosinus
from sklearn.model_selection import train_test_split  # train/val split

# Hugging Face
from datasets import load_dataset  # charger IMDB dataset
from transformers import (
    AutoTokenizer,  #tokenization
    AutoModelForSequenceClassification,  #  classification
    AutoModel,  #  embedding
    AutoModelForCausalLM,  #  text generation model
    TrainingArguments,  #  training configuration
    Trainer,  #  training the model
    pipeline
)

# LoRA fine-tuning
from peft import (
    LoraConfig,  # configuration
    get_peft_model,  # appliquer LoRA
    TaskType  # specifier task type
)

# barres de progression
from tqdm import tqdm

# PyTorch
import torch
from torch.utils.data import Dataset  # For custom datasets

print("import successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")  # Check if GPU is available


import successful
PyTorch version: 2.9.0+cu126
CUDA available: True


## etape 1.3 : charger et préparer le dataset IMDB

échantillonnage 10 000 réview : équilibrer positives/négatives



In [ ]:
# chargement données depuis Hugging Face
# movie reviews : positive/negative labels
print("Loading IMDB dataset...")
from datasets import load_dataset, concatenate_datasets # Import concatenate_datasets
dataset = load_dataset("imdb")

# charger train et test splits
original_train_data = dataset["train"]  # Original training data
original_test_data = dataset["test"]  # Original test data

# --- echantillonnage equilibré / Training Data ---
TRAIN_NEG_SAMPLES = 5000
TRAIN_POS_SAMPLES = 5000

# isoler reviews negatives et positives
train_neg = original_train_data.filter(lambda x: x['label'] == 0)
train_pos = original_train_data.filter(lambda x: x['label'] == 1)

# Selection des reviews pour l'échantillon
train_neg = train_neg.select(range(min(TRAIN_NEG_SAMPLES, len(train_neg))))
train_pos = train_pos.select(range(min(TRAIN_POS_SAMPLES, len(train_pos))))

# Combiner and mélanger les subsets
train_data = concatenate_datasets([train_neg, train_pos]) # librairie Datasets de Hugging Face
train_data = train_data.shuffle(seed=42) # librairie Datasets de Hugging Face

# --- échantillon équilibré / Test Data ---
TEST_NEG_SAMPLES = 1000
TEST_POS_SAMPLES = 1000

# isoler reviews negatives and positives des test data
test_neg = original_test_data.filter(lambda x: x['label'] == 0)
test_pos = original_test_data.filter(lambda x: x['label'] == 1)

# Selectionner l'échantillon
test_neg = test_neg.select(range(min(TEST_NEG_SAMPLES, len(test_neg))))
test_pos = test_pos.select(range(min(TEST_POS_SAMPLES, len(test_pos))))

# Combiner et mélanger l'échantillon des test_data
test_data = concatenate_datasets([test_neg, test_pos]) # cf librairie datasets de Hugging Face
test_data = test_data.shuffle(seed=42)

print(f"total extrait Training : {len(train_data)} (Negative: {TRAIN_NEG_SAMPLES}, Positive: {TRAIN_POS_SAMPLES})")
print(f"total extrait Test : {len(test_data)} (Negative: {TEST_NEG_SAMPLES}, Positive: {TEST_POS_SAMPLES})")

# afficher des exemples de review+ label
for item in range(10):

  print("\nExemples de reviews:")
  print(f"Texte: {train_data[item]['text'][:200]}...")  # premiers 200 characters
  print(f"Label: {train_data[item]['label']} (0=negative, 1=positive)")

Loading IMDB dataset...
total extrait Training : 10000 (Negative: 5000, Positive: 5000)
total extrait Test : 2000 (Negative: 1000, Positive: 1000)

Exemples de reviews:
Texte: There are many kinds of reunion shows. One kind is where old actors are taken out of mothballs and set to recreate characters they haven't played for twenty or thirty years. These have mixed results. ...
Label: 1 (0=negative, 1=positive)

Exemples de reviews:
Texte: This isn't the best romantic comedy ever made, but it is certainly pretty nice and watchable. It's directed in an old-fashioned way and that works fine. Cybill Shepherd as Corinne isn't bad in her rol...
Label: 1 (0=negative, 1=positive)

Exemples de reviews:
Texte: I've always liked Fred MacMurray, andalthough her career was tragically cut shortI think Carole Lombard is fun to watch. Pair these two major and attractive stars together, add top supporting player...
Label: 0 (0=negative, 1=positive)

Exemples de reviews:
Texte: Anna (Charlotte Burke),

# **ETAPE 2** : Modèle de classification des reviews positives (label=1) / negatives (label=0) : DistilBERT (version allégée de BERT)



## étape 2.1: charger le tokenizer et le modèle pré-entrainé


In [ ]:
# configuration du modèle
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2  # classification binaire: positive (1) or negative (0)

# charger le tokenizer pour convertir le texte de reviews en tokens (nombres)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# charger le modèle de claissification pré-entrainé
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

print("Modele et tokenizer chargés")
print(f"Paramètres du modèle: {sum(p.numel() for p in model.parameters()):,}")  # Total parameters


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modele et tokenizer chargés
Paramètres du modèle: 66,955,010


## étape 2.2: Tokenizer le Dataset

tokenizer training et test data.


In [ ]:
# Fonction : tokenizer le dataset
def tokenize_function(examples):
    """
    Tokenize  examples.
    Args: examples: Dictionary with 'text' key containing review texts
    Returns: Tokenized inputs with 'input_ids', 'attention_mask', etc.
    """
    # - truncation=True: limiter la séquenca max_length
    # - padding=True: harmoniser la longueur des séquences
    # - max_length=512: Maximulongueur max d'une sequence length (=limite de BERT)
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512
    )

# tokenization des training data
print("Tokenizing training data...")
tokenized_train = train_data.map(
    tokenize_function,
    batched=True,  # Process in batches for speed
    remove_columns=["text"]  # Remove original text (we have tokens now)
)

print("Exemple de tokenization pour les données d'entraînement:")
print(f"Original text: {train_data[0]['text'][:100]}...")
print(f"Input IDs: {tokenized_train[0]['input_ids'][:20]}...")
print(f"Attention Mask: {tokenized_train[0]['attention_mask'][:20]}...")

# Apply tokenization to test data
print("\nTokenizing test data...")
tokenized_test = test_data.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

print("Exemple de tokenization pour les données de test:")
print(f"Original text: {test_data[0]['text'][:100]}...")
print(f"Input IDs: {tokenized_test[0]['input_ids'][:20]}...")
print(f"Attention Mask: {tokenized_test[0]['attention_mask'][:20]}...")

print("\n Tokenization terminée !")
print(f"Total training: {len(tokenized_train)}")
print(f"Total test: {len(tokenized_test)}")


Tokenizing training data...
Exemple de tokenization pour les données d'entraînement:
Original text: There are many kinds of reunion shows. One kind is where old actors are taken out of mothballs and s...
Input IDs: [101, 2045, 2024, 2116, 7957, 1997, 10301, 3065, 1012, 2028, 2785, 2003, 2073, 2214, 5889, 2024, 2579, 2041, 1997, 5820]...
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]...

Tokenizing test data...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Exemple de tokenization pour les données de test:
Original text: Linda Blair has been acting for forty years now, and while she will never escape the part of Regan M...
Input IDs: [101, 8507, 10503, 2038, 2042, 3772, 2005, 5659, 2086, 2085, 1010, 1998, 2096, 2016, 2097, 2196, 4019, 1996, 2112, 1997]...
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]...

 Tokenization terminée !
Total training: 10000
Total test: 2000


## étape 2.3: Configure LoRA pour un Fine-Tuning plus rapide

réduction du nombre de paramètres à tuner


In [ ]:
# Configure LoRA parameters
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
    r=8,  # Rank: lower = fewer parameters (8 is a good balance)
    lora_alpha=16,  # Scaling factor for LoRA weights
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    target_modules=["q_lin", "v_lin"]  # Which layers to apply LoRA to
    # "q_lin" and "v_lin" are query and value linear layers in attention
)

# Apply LoRA to the model
# This freezes the base model weights and adds trainable LoRA adapters
print("Applying LoRA to model...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
# With LoRA, we only train a tiny fraction of parameters!
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print("✓ LoRA applied successfully!")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}% of total)")
print(f"Total parameters: {total_params:,}")


Applying LoRA to model...
✓ LoRA applied successfully!
Trainable parameters: 739,586 (1.09% of total)
Total parameters: 67,694,596


## étape 2.4: mise en place du process de training

We configure the training process: learning rate, batch size, number of epochs, etc.


In [ ]:
# Training arguments
# These control how the model is trained
training_args = TrainingArguments(
    output_dir="./results",  # Where to save model checkpoints
    num_train_epochs=2, #3,  # Number of training epochs (full passes through data)
    per_device_train_batch_size=8,  # Batch size per device (small for memory)
    per_device_eval_batch_size=8,  # Evaluation batch size
    learning_rate=2e-4,  # Learning rate (how fast model learns)
    weight_decay=0.01,  # Regularization to prevent overfitting
    logging_dir="./logs",  # Where to save logs
    logging_steps=100,  # Log every 100 steps
    eval_strategy="epoch",  # Evaluate at end of each epoch
    save_strategy="epoch",  # Save model at end of each epoch
    load_best_model_at_end=True,  # Load best model after training
    fp16=True,  # Use mixed precision (faster, less memory) if GPU available
    report_to="none"  # Don't report to external services
)

print("✓ Training configuration set up!")


✓ Training configuration set up!


## étape 2.5: split entre train et validation


We split the training data into train and validation sets, then create a Trainer object to handle the training process.


In [ ]:
## NE PAS REFAIRE LE SPLIT : PARTIE A EFFACER/DEBUT
# # Split tokenized training data into train and validation sets
# # 80% for training, 20% for validation
# train_val_split = tokenized_train.train_test_split(test_size=0.2, seed=42)

# train_dataset = train_val_split["train"]  # Training set
# val_dataset = train_val_split["test"]  # Validation set
###PARTIE A EFFACER/FIN

# attribution des données pour le training et le test à partir de l'échantillon
#  équilibré extrait du dataset IMDB au début
train_dataset = tokenized_train
val_dataset = tokenized_test

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Create a metric function for evaluation
# This computes accuracy during training
def compute_metrics(eval_pred):
    """
    Compute accuracy metric.

    Args:
        eval_pred: Tuple of (predictions, labels)

    Returns:
        Dictionary with accuracy score
    """
    predictions, labels = eval_pred
    # Get predicted class (0 or 1) from logits
    predictions = np.argmax(predictions, axis=1)
    # Calculate accuracy
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Create the Trainer
# Trainer handles all the training loop, evaluation, and saving
trainer = Trainer(
    model=model,  # The model to train
    args=training_args,  # Training configuration
    train_dataset=train_dataset,  # Training data
    eval_dataset=val_dataset,  # Validation data
    compute_metrics=compute_metrics,  # How to compute metrics
    tokenizer=tokenizer  # Tokenizer for decoding
)

print("✓ Trainer created and ready for training!")


Training samples: 10000
Validation samples: 2000


/tmp/ipython-input-1413572791.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✓ Trainer created and ready for training!


## Step 3: Train the Model

Now we train the model! This may take some time depending on your hardware. With LoRA, it should be faster than full fine-tuning.


In [ ]:
# Train the model
# This will take several minutes depending on your hardware
print("Starting training...")
print("This may take a few minutes. Please wait...")

# Start training
trainer.train()

print("✓ Training complete!")

# Evaluate on validation set
print("\nEvaluating on validation set...")
eval_results = trainer.evaluate()
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")


Starting training...
This may take a few minutes. Please wait...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.285800,0.255849,0.900500
2,0.216600,0.260961,0.910500


✓ Training complete!

Evaluating on validation set...


Validation Accuracy: 0.9005
Validation Loss: 0.2558


## Step 4: Save the Fine-Tuned Model

We save the trained model so we can use it later without retraining.


In [ ]:
# Save the fine-tuned model
MODEL_SAVE_PATH = "./fine_tuned_sentiment_model"

print(f"Saving model to {MODEL_SAVE_PATH}...")
trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print("✓ Model saved successfully!")

# Also save the full model with LoRA adapters
# This allows us to load it later
model.save_pretrained(MODEL_SAVE_PATH)


Saving model to ./fine_tuned_sentiment_model...
✓ Model saved successfully!


## Step 5: Test the Sentiment Classifier

Let's test our fine-tuned model on some example reviews to see if it works correctly.


In [ ]:
# Create a sentiment analysis pipeline for easy inference
# This makes it easy to classify new text
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# Test on some example reviews
test_reviews = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Terrible movie, waste of time. Boring and poorly acted.",
    "It was okay, nothing special but not bad either.",
    "One of the best films I've ever seen. Highly recommended!",
    "Awful. Just awful. Don't watch this."
]

print("Testing sentiment classifier:\n")
for review in test_reviews:
    result = sentiment_pipeline(review)[0]
    label = "POSITIVE" if result["label"] == "LABEL_1" else "NEGATIVE"
    score = result["score"]
    print(f"Review: {review[:60]}...")
    print(f"  → {label} (confidence: {score:.3f})\n")


Device set to use cuda:0


Testing sentiment classifier:

Review: This movie was absolutely fantastic! I loved every minute of...
  → POSITIVE (confidence: 0.998)

Review: Terrible movie, waste of time. Boring and poorly acted....
  → NEGATIVE (confidence: 0.999)

Review: It was okay, nothing special but not bad either....
  → POSITIVE (confidence: 0.558)

Review: One of the best films I've ever seen. Highly recommended!...
  → POSITIVE (confidence: 0.996)

Review: Awful. Just awful. Don't watch this....
  → NEGATIVE (confidence: 0.987)



## Step 6: Build the RAG System - Prepare Corpus and Embeddings

Now we build the RAG (Retrieval-Augmented Generation) system. This will:
1. Create embeddings of all reviews using BERT
2. Integration of normalization in the custom via "normalize_embeddings" attribute
3. Implement kNN search for retrieval


In [ ]:
# --- ÉTAPE 1 : Remplacement du modèle d'embedding ---
# On utilise Sentence-BERT, spécifiquement conçu pour la similarité sémantique.
# 'all-MiniLM-L6-v2' est un excellent standard, rapide et performant.
# 'all-mpnet-base-v2' est plus précis mais plus lent (à tester si besoin).
from sentence_transformers import SentenceTransformer
import numpy as np # S'assurer que numpy est importé

embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
print(f"Chargement du modèle d'embedding optimisé : {embedding_model_name}")
embedding_model = SentenceTransformer(embedding_model_name)
# Plus besoin de tokenizer séparé, SentenceTransformer gère tout.

# Passage sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = embedding_model.to(device)
print(f"✓ Modèle d'embedding chargé sur {device}")

# --- Mise à jour de la fonction get_embeddings ---
def get_embeddings(texts, batch_size=32):
    """Generate embeddings using SentenceTransformer (optimisé)."""
    if isinstance(texts, str):
        texts = [texts]

    # .encode() gère le tokenizing, le padding, le pooling et la normalisation
    # On utilise show_progress_bar=True pour suivre l'avancement
    embeddings = embedding_model.encode(
        texts,
        batch_size=batch_size,
        convert_to_tensor=False, # On veut des numpy arrays pour FAISS
        show_progress_bar=True,
        device=device, # Utiliser le bon device
        normalize_embeddings=True # <--- OPTIMISATION : Normalisation L2 directe ici !
    )

    # Si le résultat est un tenseur, on le convertit en numpy
    if isinstance(embeddings, torch.Tensor):
        embeddings = embeddings.cpu().numpy()

    # On s'assure que c'est du float32 (standard pour FAISS et calculs vectoriels)
    return embeddings.astype(np.float32)

print("✓ Nouvelle fonction get_embeddings prête avec Sentence-BERT + Normalisation.")

# Get all review texts from training data
print("\nExtracting review texts...")
# Assurez-vous que train_data est défini avant ce bloc
corpus_texts = [item["text"] for item in train_data]
print(f"Corpus size: {len(corpus_texts)} reviews")

# Create embeddings for the corpus
print("\nCreating Sentence embeddings (this may take a few minutes)...")
# On utilise la nouvelle fonction get_embeddings
corpus_embeddings = get_embeddings(corpus_texts)

print(f"✓ Embeddings created! Shape: {corpus_embeddings.shape}")

Chargement du modèle d'embedding optimisé : sentence-transformers/all-MiniLM-L6-v2
✓ Modèle d'embedding chargé sur cuda
✓ Nouvelle fonction get_embeddings prête avec Sentence-BERT + Normalisation.

Extracting review texts...
Corpus size: 10000 reviews

Creating Sentence embeddings (this may take a few minutes)...


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

✓ Embeddings created! Shape: (10000, 384)


In [ ]:
import re  # Import pour les expressions régulières (nettoyage)

# Note: On a supprimé l'étape TF-IDF qui était redondante.
# Le système repose entièrement sur la recherche sémantique (Embeddings + kNN).

# Build kNN index using cosine similarity
# kNN finds the most similar documents to a query
print("\nBuilding kNN index...")
knn_model = NearestNeighbors(
    n_neighbors=10,  # Retrieve top 10 most similar documents
    metric="cosine"  # Use cosine similarity (measures angle between vectors)
)
knn_model.fit(corpus_embeddings)  # Fit on BERT embeddings
print("✓ kNN index built!")

# Function to retrieve relevant documents
def retrieve_documents(query_text, top_k=5):
    """
    Retrieve top-k most similar documents to a query.
    Args:
        query_text: The query text
        top_k: Number of documents to retrieve
    Returns:
        List of dictionaries with text, similarity, and sentiment
    """
    # Get query embedding
    query_embedding = get_embeddings([query_text])

    # Find nearest neighbors
    distances, indices = knn_model.kneighbors(query_embedding, n_neighbors=top_k)

    # Get retrieved documents
    retrieved_docs = []
    for idx, dist in zip(indices[0], distances[0]):
        raw_text = corpus_texts[idx]

        # --- OPTIMISATION 1 : Récupération du Sentiment ---
        # On récupère le label d'origine (0 ou 1) via l'index
        # (Suppose que train_data est toujours disponible)
        label_id = train_data[int(idx)]['label']
        sentiment_str = "POSITIVE" if label_id == 1 else "NEGATIVE"

        # --- NETTOYAGE DU TEXTE ---
        clean_text = re.sub(r'<[^>]+>', ' ', raw_text)
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()

        retrieved_docs.append({
            "text": clean_text,
            "similarity": 1 - dist,
            "sentiment": sentiment_str # Ajout de la métadonnée
        })

    return retrieved_docs

# Test retrieval
print("\nTesting retrieval system (Optimized)...")
test_query = "I love action movies with great special effects"
retrieved = retrieve_documents(test_query, top_k=3)
print(f"\nQuery: {test_query}")
print("\nRetrieved documents:")
for i, doc in enumerate(retrieved, 1):
    print(f"{i}. [{doc['sentiment']}] Similarity: {doc['similarity']:.3f}")
    print(f"   Text: {doc['text'][:100]}...")


Building kNN index...
✓ kNN index built!

Testing retrieval system (Optimized)...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: I love action movies with great special effects

Retrieved documents:
1. [POSITIVE] Similarity: 0.627
   Text: I'm stunt, I must admit I never saw a movie with such good story and none stop high special effect m...
2. [POSITIVE] Similarity: 0.546
   Text: Police Story is arguably one of the best works by the master of action himself.Compared to other act...
3. [NEGATIVE] Similarity: 0.541
   Text: There is a really good movie lurking just beneath the surface of the layers upon layers of cheese th...


In [ ]:
# Test de récupération (Retrieval) avec le modèle kNN
query_test = "The special effects were amazing and the plot was thrilling"

print(f"🔎 Requête : '{query_test}'")
print("-" * 50)

# Appel de la fonction de récupération qui utilise l'index kNN
# Elle convertit la requête en vecteur et cherche les voisins les plus proches
retrieved_docs = retrieve_documents(query_test, top_k = 10)

# Affichage des résultats
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n📄 Résultat #{i}")
    print(f"   Score de similarité (Cosinus) : {doc['similarity']:.4f}")
    print(f"   Extrait du texte : \"{doc['text'][:250]}...\"")

🔎 Requête : 'The special effects were amazing and the plot was thrilling'
--------------------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📄 Résultat #1
   Score de similarité (Cosinus) : 0.6535
   Extrait du texte : "There is a really good movie lurking just beneath the surface of the layers upon layers of cheese that is "the Pagemaster". I found this out when I watched this again this morning after neglecting it for years and years. I remember hating it with a p..."

📄 Résultat #2
   Score de similarité (Cosinus) : 0.6503
   Extrait du texte : "Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and porno sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoya..."

📄 Résultat #3
   Score de similarité (Cosinus) : 0.6426
   Extrait du texte : "Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoyable mo

## Step 6.5: Save Corpus Data for CLI Use

We save the corpus texts and embeddings so they can be loaded by the CLI interface script.


In [ ]:
# Save corpus texts and embeddings for CLI use
# This allows the CLI script to load the data without recomputing embeddings
print("Saving corpus data for CLI use...")

# Save corpus texts as numpy array (for easy loading)
np.save("corpus_texts.npy", np.array(corpus_texts, dtype=object))

# Save embeddings
np.save("corpus_embeddings.npy", corpus_embeddings)

print("✓ Corpus data saved!")
print("  - corpus_texts.npy")
print("  - corpus_embeddings.npy")
print("\nThese files can now be loaded by the CLI interface script.")


Saving corpus data for CLI use...
✓ Corpus data saved!
  - corpus_texts.npy
  - corpus_embeddings.npy

These files can now be loaded by the CLI interface script.


## Step 7 Custom du modèle de génération de texte à partir du modèle entrainé

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load Flan-T5 for text generation
# Flan-T5 is an instruction-tuned model excellent for RAG tasks.
print("Loading Flan-T5 for text generation...")
generator_model_name = "google/flan-t5-base"
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = AutoModelForSeq2SeqLM.from_pretrained(generator_model_name)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator_model = generator_model.to(device)
generator_model.eval()  # Evaluation mode

print(f"✓ Generator model loaded on {device}")

# Function to generate response using retrieved context
def generate_response(
    query,
    retrieved_docs,
    max_new_tokens=150,
    num_context_docs=2,
    max_doc_length=600
):
    # 1. Prepare Context WITHOUT Labels
    context_parts = []
    for doc in retrieved_docs[:num_context_docs]:
        # Take up to max_doc_length characters from each document
        context_parts.append(f"Review: {doc['text'][:max_doc_length]}")

    context = "\n\n".join(context_parts)

    # 2. Construct Optimized Prompt
    # On donne un rôle clair à l'IA et des instructions précises
    if not context:
        # Special instruction if no context is available
        prompt = f"""You are an expert movie assistant. You were asked: '{query}'. However, no relevant reviews were found to answer this question. Please state that you cannot provide an answer based on available information.

Answer:"""
    else:
        prompt = f"""You are an expert movie assistant. Use the reviews below to answer the user's question.
If the reviews are negative, explain why. If positive, highlight the good points.
If the provided reviews do not contain enough information to fully answer the question, state that you cannot answer completely based on the given context.

Context Reviews:
{context}

User Question: {query}

Answer:"""

    # Tokenize prompt
    inputs = generator_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Generate response
    with torch.no_grad():
        outputs = generator_model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            min_length=30,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            no_repeat_ngram_size=2
        )

    # Decode generated text
    generated_text = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text.strip()

Loading Flan-T5 for text generation...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✓ Generator model loaded on cuda


## Step 7.5 Test Modele Génération de code

In [2]:
# Test generation with IMPROVED settings
print("\nTesting text generation with Flan-T5 (IMPROVED)...\n")
test_query = "What do people think about the special effects in avatar ?"

# Retrieval (now includes sentiment)
retrieved = retrieve_documents(test_query, top_k=5)

# Generate response (uses optimized prompt with sentiment context)
response = generate_response(test_query, retrieved)

print(f"Query: {test_query}")
print(f"\nGenerated Response (Length: {len(response.split())} words):")
print("-" * 50)
print(response)
print("-" * 50)


Testing text generation with Flan-T5 (IMPROVED)...



NameError: name 'retrieve_documents' is not defined

## Step 8 Sauvegarde du modèle

In [ ]:
# Sauvegarde du modèle d'embeddings
# Note: SentenceTransformer gère son propre tokenizer interne et utilise .save()
MODEL_EMBEDDING_PATH = "./embedding_model"

print(f"Saving embedding model to {MODEL_EMBEDDING_PATH}...")
embedding_model.save(MODEL_EMBEDDING_PATH)
print("✓ Embedding model saved successfully!")

# Sauvegarde du modèle de génération et son tokenizer
MODEL_GENERATOR_PATH = "./generator_model"

print(f"Saving generator model to {MODEL_GENERATOR_PATH}...")
generator_model.save_pretrained(MODEL_GENERATOR_PATH)
generator_tokenizer.save_pretrained(MODEL_GENERATOR_PATH)
print("✓ Generator model saved successfully!")

Saving embedding model to ./embedding_model...
✓ Embedding model saved successfully!
Saving generator model to ./generator_model...
✓ Generator model saved successfully!


In [ ]:
# import shutil
# import os

# # Define paths for the fine-tuned sentiment model and its tokenizer
# MODEL_SAVE_PATH = "./fine_tuned_sentiment_model"

# # 1. Compresser le dossier du modèle de sentiment affiné
# model_dir = MODEL_SAVE_PATH
# zip_file_name_sentiment = "fine_tuned_sentiment_model.zip"

# # Créer l'archive zip
# print(f"Compression du dossier '{model_dir}' en '{zip_file_name_sentiment}'...")
# shutil.make_archive(model_dir, 'zip', model_dir)
# print("✓ Dossier du modèle de sentiment compressé avec succès !")

# # 2. Télécharger le fichier zip du modèle de sentiment
# from google.colab import files

# print(f"Téléchargement de '{zip_file_name_sentiment}'...")
# files.download(zip_file_name_sentiment)
# print("✓ Téléchargement du modèle de sentiment lancé !")


# # Compresser le dossier du modèle d'embeddings
# embedding_model_dir = './embedding_model'
# embedding_zip_file_name = "embedding_model.zip"
# print(f"Compression du dossier '{embedding_model_dir}' en '{embedding_zip_file_name}'...")
# shutil.make_archive(embedding_model_dir, 'zip', embedding_model_dir)
# print("✓ Dossier d'embeddings compressé avec succès !")

# # Compresser le dossier du modèle de génération
# generator_model_dir = './generator_model'
# generator_zip_file_name = "generator_model.zip"
# print(f"Compression du dossier '{generator_model_dir}' en '{generator_zip_file_name}'...")
# shutil.make_archive(generator_model_dir, 'zip', generator_model_dir)
# print("✓ Dossier de génération compressé avec succès !")

# # Télécharger les fichiers zip des modèles d'embedding et de génération
# print(f"Téléchargement de '{embedding_zip_file_name}'...")
# files.download(embedding_zip_file_name)

# print(f"Téléchargement de '{generator_zip_file_name}'...")
# files.download(generator_zip_file_name)

# # Télécharger les fichiers corpus_embeddings.npy et corpus_texts.npy
# print("Téléchargement de 'corpus_embeddings.npy'...")
# files.download('corpus_embeddings.npy')

# print("Téléchargement de 'corpus_texts.npy'...")
# files.download('corpus_texts.npy')

# print("✓ Téléchargements lancés !")

Compression du dossier './fine_tuned_sentiment_model' en 'fine_tuned_sentiment_model.zip'...
✓ Dossier du modèle de sentiment compressé avec succès !
Téléchargement de 'fine_tuned_sentiment_model.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Téléchargement du modèle de sentiment lancé !
Compression du dossier './embedding_model' en 'embedding_model.zip'...
✓ Dossier d'embeddings compressé avec succès !
Compression du dossier './generator_model' en 'generator_model.zip'...
✓ Dossier de génération compressé avec succès !
Téléchargement de 'embedding_model.zip'...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Téléchargement de 'corpus_texts.npy'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Téléchargements lancés !


## Step 9: Create Evaluation Functions

We create functions to evaluate the system on multiple metrics: accuracy, response length, repetitions, keyword presence, and latency.


In [ ]:
# Evaluation functions

def count_repetitions(text, n=3):
    """
    Count repeated n-grams (phrases) in text.
    Higher repetition = lower quality.

    Args:
        text: Text to analyze
        n: Length of n-grams to check

    Returns:
        Number of repeated n-grams
    """
    words = text.lower().split()
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    ngram_counts = Counter(ngrams)
    # Count n-grams that appear more than once
    repetitions = sum(1 for count in ngram_counts.values() if count > 1)
    return repetitions

def check_keyword_presence(text, keywords):
    """
    Check if important keywords from query appear in response.

    Args:
        text: Response text
        keywords: List of keywords to check

    Returns:
        Number of keywords found
    """
    text_lower = text.lower()
    found = sum(1 for keyword in keywords if keyword.lower() in text_lower)
    return found

def evaluate_response(query, response, true_label=None, start_time=None):
    """
    Evaluate a single response on multiple metrics.

    Args:
        query: Original query
        response: Generated response
        true_label: True sentiment label (if available)
        start_time: Start time for latency calculation

    Returns:
        Dictionary with evaluation metrics
    """
    metrics = {}

    # Response length
    metrics["response_length"] = len(response.split())

    # Repetitions
    metrics["repetitions"] = count_repetitions(response)

    # Keyword presence (extract important words from query)
    query_words = [w for w in query.lower().split() if len(w) > 3]  # Words longer than 3 chars
    metrics["keywords_found"] = check_keyword_presence(response, query_words)

    # Latency (time taken to generate)
    if start_time:
        metrics["latency"] = time.time() - start_time

    # Classification accuracy (if true label provided)
    if true_label is not None:
        result = sentiment_pipeline(query)[0]
        predicted_label = 1 if result["label"] == "LABEL_1" else 0
        metrics["classification_correct"] = (predicted_label == true_label)
        metrics["classification_confidence"] = result["score"]

    return metrics

print("✓ Evaluation functions created!")


✓ Evaluation functions created!


## Step 10: Run End-to-End Evaluation

We test the complete system on 50 queries and collect all evaluation metrics.


In [ ]:
# Create test queries for evaluation
# Mix of different types of queries
test_queries = [
    "What did people think about the plot?",
    "How was the acting?",
    "Was the movie entertaining?",
    "Did people like the special effects?",
    "What about the cinematography?",
    "How was the dialogue?",
    "Was the movie well-directed?",
    "Did people enjoy the soundtrack?",
    "How was the pacing?",
    "What did people think about the ending?",
] * 5 # On fait un test rapide sur 10 requêtes (sans multiplier par 5)

# Run evaluation
print(f"Running evaluation on {len(test_queries)} queries (Quick Test)...")
print("This may take a minute...\n")

# --- FIX: Ensure sentiment_pipeline is defined ---
try:
    sentiment_pipeline
except NameError:
    print("sentiment_pipeline not found. Initializing it now...")
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
# -----------------------------------------------

results = []

for i, query in enumerate(tqdm(test_queries, desc="Evaluating")):
    start_time = time.time()

    # Get sentiment classification
    sentiment_result = sentiment_pipeline(query)[0]
    predicted_sentiment = "POSITIVE" if sentiment_result["label"] == "LABEL_1" else "NEGATIVE"

    # Retrieve relevant documents
    retrieved = retrieve_documents(query, top_k=5)

    # Generate response
    response = generate_response(query, retrieved)

    # Evaluate
    metrics = evaluate_response(query, response, start_time=start_time)

    # Store results
    results.append({
        "query": query,
        "sentiment": predicted_sentiment,
        "sentiment_confidence": sentiment_result["score"],
        "response": response,
        **metrics  # Add all evaluation metrics
    })

# Convert to DataFrame for easy analysis
results_df = pd.DataFrame(results)

# Calculate summary statistics
print("\n" + "="*60)
print("EVALUATION RESULTS (Optimized System)")
print("="*60)
print(f"\nTotal queries: {len(results_df)}")
print(f"Average response length: {results_df['response_length'].mean():.1f} words")
print(f"Average repetitions: {results_df['repetitions'].mean():.1f}")
print(f"Average keywords found: {results_df['keywords_found'].mean():.1f}")
print(f"Average latency: {results_df['latency'].mean():.2f} seconds")
print(f"\nSentiment distribution:")
print(results_df['sentiment'].value_counts())

# Save results to CSV
results_df.to_csv("evaluation_results_optimized.csv", index=False)
print("\n✓ Results saved to 'evaluation_results_optimized.csv'")

Running evaluation on 50 queries (Quick Test)...
This may take a minute...



Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   2%|▏         | 1/50 [00:00<00:45,  1.08it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:  98%|█████████▊| 49/50 [00:51<00:00,  1.17it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


EVALUATION RESULTS (Optimized System)

Total queries: 50
Average response length: 32.5 words
Average repetitions: 0.0
Average keywords found: 0.4
Average latency: 1.06 seconds

Sentiment distribution:
sentiment
NEGATIVE    45
POSITIVE     5
Name: count, dtype: int64

✓ Results saved to 'evaluation_results_optimized.csv'


## Step 11: Robustness Tests

We test the system with edge cases: vague prompts, contradictory prompts, and unrelated prompts.


In [ ]:
# Robustness tests
print("="*60)
print("ROBUSTNESS TESTS")
print("="*60)

# Test 1: Vague prompt
print("\n1. VAGUE PROMPT TEST")
print("-" * 60)
vague_query = "What about it?"
print(f"Query: '{vague_query}'")
retrieved = retrieve_documents(vague_query, top_k=3)
response = generate_response(vague_query, retrieved)
print(f"Response: {response[:200]}...")
metrics = evaluate_response(vague_query, response)
print(f"Metrics: Length={metrics['response_length']}, Repetitions={metrics['repetitions']}")

# Test 2: Contradictory prompt
print("\n2. CONTRADICTORY PROMPT TEST")
print("-" * 60)
contradictory_query = "This movie is both amazing and terrible at the same time"
print(f"Query: '{contradictory_query}'")
sentiment_result = sentiment_pipeline(contradictory_query)[0]
print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.3f})")
retrieved = retrieve_documents(contradictory_query, top_k=3)
response = generate_response(contradictory_query, retrieved)
print(f"Response: {response[:200]}...")

# Test 3: Unrelated prompt
print("\n3. UNRELATED PROMPT TEST")
print("-" * 60)
unrelated_query = "What is the weather like today?"
print(f"Query: '{unrelated_query}'")
retrieved = retrieve_documents(unrelated_query, top_k=3)
print(f"Retrieved documents similarity scores:")
for i, doc in enumerate(retrieved, 1):
    print(f"  {i}. {doc['similarity']:.3f}")
response = generate_response(unrelated_query, retrieved)
print(f"Response: {response[:200]}...")

print("\n" + "="*60)
print("Robustness tests complete!")
print("="*60)


ROBUSTNESS TESTS

1. VAGUE PROMPT TEST
------------------------------------------------------------
Query: 'What about it?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Response: negative. - The story is a joke and very stupid.- It is bad in every aspect. This movie is not funny. I didn't like it....
Metrics: Length=25, Repetitions=0

2. CONTRADICTORY PROMPT TEST
------------------------------------------------------------
Query: 'This movie is both amazing and terrible at the same time'
Sentiment: LABEL_1 (confidence: 0.958)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Response: Not enough information about this movie. It has no redeeming value. This movie is very boring! It isn't worth seeing. If you can't even finish the movie, you'll lose your interest....

3. UNRELATED PROMPT TEST
------------------------------------------------------------
Query: 'What is the weather like today?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved documents similarity scores:
  1. 0.261
  2. 0.235
  3. 0.228
Response: It's rainy and cloudy. The weather is cloudier than usual. It is sunny and cold. A lot of the characters aren't very smart....

Robustness tests complete!


## Step 12: Summary and Reflection

### Model Choices

**DistilBERT for Classification:**
- Smaller and faster than BERT
- Good performance for binary classification
- LoRA fine-tuning makes it efficient

**DistilGPT-2 for Generation:**
- Lightweight text generation model
- Fast inference
- Works well with retrieved context

### TF-IDF + kNN Pipeline

**TF-IDF (Term Frequency-Inverse Document Frequency):**
- Converts text to numerical vectors
- Highlights important words
- Fast and memory-efficient

**kNN (k-Nearest Neighbors):**
- Finds most similar documents using cosine similarity
- Works with BERT embeddings
- Simple and effective for retrieval

### CPU Limitations

- Training and inference can be slow on CPU
- GPU significantly speeds up:
  - Model training
  - Embedding creation
  - Text generation
- Google Colab provides free GPU access

### CLI Simplicity

- Interactive loop design
- Easy to use for beginners
- Integrates all components
- Collects feedback automatically

### Robustness Observations

1. **Vague prompts**: System may struggle with unclear queries
2. **Contradictory prompts**: Sentiment classifier picks one direction
3. **Unrelated prompts**: Retrieval may find loosely related documents

### Future Improvements

- Larger training dataset
- Better prompt engineering
- Fine-tune generator on movie reviews
- Use more sophisticated retrieval (e.g., dense passage retrieval)
- Implement better repetition detection and prevention
